In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def batch_canny(images):
    canny_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        canny_superposition = np.zeros(shape=(img.shape[0],img.shape[1]),dtype=np.uint16)
        for channel in range(3):
            img_ch = img[:,:,channel]
            canny_ch = cv2.Canny(img_ch, 60, 150)
            canny_superposition =  canny_superposition + (canny_ch).astype(np.uint16)
        canny_superposition = cv2.normalize(canny_superposition, canny_superposition,0,255,cv2.NORM_MINMAX)
        canny_list.append(np.array((canny_superposition).astype(np.uint8)).reshape([img.shape[0],img.shape[1],1]))
    canny_list = np.array(canny_list,dtype = float32)
    return canny_list


In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
#     train_images_original = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
#     train_images_sub = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]
#     train_images = train_images_original + train_images_sub
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

trainImage_canny = batch_canny(trainImage)
testImage_canny = batch_canny(testImage)

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)
np.save('../trainImage_canny.npy', trainImage_canny)
np.save('../testImage_canny.npy', testImage_canny)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] trainImage_canny matrix: {:.2f}MB".format(
    (trainImage_canny.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny matrix: {:.4f}MB".format(
    (testImage_canny.nbytes) / (1024 * 1000.0)))

[INFO] handling images...
[INFO] processed 1000/4000
[INFO] processed 2000/4000
[INFO] processed 3000/4000
[INFO] processed 4000/4000
[INFO] processed 156/156
(4000, 128, 128, 3)


100%|██████████| 156/156 [00:01<00:00, 80.12it/s]


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0.

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_CANNY_C%d_D%d_Kernel(%d,%d)' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1])    

        # train
        #设置一个全局的计数器
#         self.global_step = tf.Variable(0, trainable=False)
#        self.lr = tf.train.exponential_decay(0.0002,  self.global_step, 500, 0.95, staircase=True)
        self.lr = 0.001
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        self.train_x_canny = np.load('../trainImage_canny.npy')
        self.test_x_canny = np.load('../testImage_canny.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_canny, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_canny",x_canny.get_shape()) # 128, 128, 1
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_canny,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_canny = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, 1], 
                                name='x_canny')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_canny = self.train_x_canny[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_canny = shuffled_set_canny[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_canny: batch_x_canny,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
                    # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_canny: batch_x_canny_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
        
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

                # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
#    Avg per epoch ptime: 10.05, total 50 epochs ptime: 523.65
#  [*] Training finished!
#  [*] Best Epoch:  3 , Accuracy:  0.8799999952316284
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_CANNY_C5_D1_Kernel(3,3)/CNN_CANNY_C5_D1_Kernel(3,3)-3
#  [*] Finished testing Best Epoch: 3 , accuracy:  0.8799999952316284 !


CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x1x32) [288, bytes: 1152]
cnn/conv_1

Epoch: [ 3] [  19/  40] time: 28.5320, loss: 0.06021196
Epoch: [ 3] [  20/  40] time: 28.7671, loss: 0.03430269
Epoch: [ 3] [  21/  40] time: 29.0023, loss: 0.02499044
Epoch: [ 3] [  22/  40] time: 29.2365, loss: 0.04367264
Epoch: [ 3] [  23/  40] time: 29.4714, loss: 0.02704263
Epoch: [ 3] [  24/  40] time: 29.7056, loss: 0.01596416
Epoch: [ 3] [  25/  40] time: 29.9410, loss: 0.03019546
Epoch: [ 3] [  26/  40] time: 30.1758, loss: 0.02261251
Epoch: [ 3] [  27/  40] time: 30.4116, loss: 0.04230334
Epoch: [ 3] [  28/  40] time: 30.6462, loss: 0.02593972
Epoch: [ 3] [  29/  40] time: 30.8813, loss: 0.01395848
Epoch: [ 3] [  30/  40] time: 31.1196, loss: 0.01984701
Epoch: [ 3] [  31/  40] time: 31.3577, loss: 0.01142183
Epoch: [ 3] [  32/  40] time: 31.5929, loss: 0.00779596
Epoch: [ 3] [  33/  40] time: 31.8274, loss: 0.04454654
Epoch: [ 3] [  34/  40] time: 32.0641, loss: 0.05497654
Epoch: [ 3] [  35/  40] time: 32.3009, loss: 0.00817303
Epoch: [ 3] [  36/  40] time: 32.5354, loss: 0.0

Epoch: [ 7] [   2/  40] time: 66.1576, loss: 0.00409807
Epoch: [ 7] [   3/  40] time: 66.3941, loss: 0.01304581
Epoch: [ 7] [   4/  40] time: 66.6297, loss: 0.00682496
Epoch: [ 7] [   5/  40] time: 66.8655, loss: 0.00943108
Epoch: [ 7] [   6/  40] time: 67.1016, loss: 0.00845080
Epoch: [ 7] [   7/  40] time: 67.3378, loss: 0.02088985
Epoch: [ 7] [   8/  40] time: 67.5738, loss: 0.00644537
Epoch: [ 7] [   9/  40] time: 67.8100, loss: 0.00549367
Epoch: [ 7] [  10/  40] time: 68.0474, loss: 0.01016202
Epoch: [ 7] [  11/  40] time: 68.2887, loss: 0.00319135
Epoch: [ 7] [  12/  40] time: 68.5241, loss: 0.01030468
Epoch: [ 7] [  13/  40] time: 68.7588, loss: 0.03085759
Epoch: [ 7] [  14/  40] time: 68.9955, loss: 0.00262035
Epoch: [ 7] [  15/  40] time: 69.2309, loss: 0.00644938
Epoch: [ 7] [  16/  40] time: 69.4684, loss: 0.00530318
Epoch: [ 7] [  17/  40] time: 69.7031, loss: 0.00226249
Epoch: [ 7] [  18/  40] time: 69.9414, loss: 0.01920005
Epoch: [ 7] [  19/  40] time: 70.1768, loss: 0.0

Epoch: [10] [  26/  40] time: 103.0780, loss: 0.00246223
Epoch: [10] [  27/  40] time: 103.3138, loss: 0.02246504
Epoch: [10] [  28/  40] time: 103.5490, loss: 0.00113506
Epoch: [10] [  29/  40] time: 103.7858, loss: 0.00083900
Epoch: [10] [  30/  40] time: 104.0227, loss: 0.00917874
Epoch: [10] [  31/  40] time: 104.2589, loss: 0.03520655
Epoch: [10] [  32/  40] time: 104.4961, loss: 0.00267078
Epoch: [10] [  33/  40] time: 104.7318, loss: 0.02139476
Epoch: [10] [  34/  40] time: 104.9691, loss: 0.01236168
Epoch: [10] [  35/  40] time: 105.2047, loss: 0.00998389
Epoch: [10] [  36/  40] time: 105.4421, loss: 0.00595106
Epoch: [10] [  37/  40] time: 105.6774, loss: 0.02624635
Epoch: [10] [  38/  40] time: 105.9138, loss: 0.00272295
Epoch: [10] [  39/  40] time: 106.1511, loss: 0.00230544
[10/50] - ptime: 10.0039 loss: 0.01576633 acc: 0.82000
Epoch: [11] [   0/  40] time: 107.3308, loss: 0.01598275
Epoch: [11] [   1/  40] time: 107.5677, loss: 0.00582716
Epoch: [11] [   2/  40] time: 107

Epoch: [14] [   6/  40] time: 140.0192, loss: 0.02583711
Epoch: [14] [   7/  40] time: 140.2554, loss: 0.00642702
Epoch: [14] [   8/  40] time: 140.4924, loss: 0.00429023
Epoch: [14] [   9/  40] time: 140.7283, loss: 0.00472482
Epoch: [14] [  10/  40] time: 140.9666, loss: 0.00188809
Epoch: [14] [  11/  40] time: 141.2036, loss: 0.00035437
Epoch: [14] [  12/  40] time: 141.4417, loss: 0.02144909
Epoch: [14] [  13/  40] time: 141.6783, loss: 0.04104402
Epoch: [14] [  14/  40] time: 141.9136, loss: 0.00016959
Epoch: [14] [  15/  40] time: 142.1523, loss: 0.00243199
Epoch: [14] [  16/  40] time: 142.3886, loss: 0.00471053
Epoch: [14] [  17/  40] time: 142.6254, loss: 0.00041296
Epoch: [14] [  18/  40] time: 142.8620, loss: 0.00237658
Epoch: [14] [  19/  40] time: 143.0995, loss: 0.00647125
Epoch: [14] [  20/  40] time: 143.3358, loss: 0.00006609
Epoch: [14] [  21/  40] time: 143.5754, loss: 0.00806849
Epoch: [14] [  22/  40] time: 143.8127, loss: 0.00008752
Epoch: [14] [  23/  40] time: 1

...

Epoch: [35] [   9/  40] time: 359.7683, loss: 0.00005654
Epoch: [35] [  10/  40] time: 360.0089, loss: 0.00004831
Epoch: [35] [  11/  40] time: 360.2448, loss: 0.00000111
Epoch: [35] [  12/  40] time: 360.4809, loss: 0.00000297
Epoch: [35] [  13/  40] time: 360.7195, loss: 0.00080629
Epoch: [35] [  14/  40] time: 360.9607, loss: 0.00002151
Epoch: [35] [  15/  40] time: 361.1975, loss: 0.00023505
Epoch: [35] [  16/  40] time: 361.4339, loss: 0.00000581
Epoch: [35] [  17/  40] time: 361.6701, loss: 0.00000160
Epoch: [35] [  18/  40] time: 361.9083, loss: 0.00000800
Epoch: [35] [  19/  40] time: 362.1461, loss: 0.00000072
Epoch: [35] [  20/  40] time: 362.3826, loss: 0.00003493
Epoch: [35] [  21/  40] time: 362.6197, loss: 0.00000891
Epoch: [35] [  22/  40] time: 362.8564, loss: 0.00000111
Epoch: [35] [  23/  40] time: 363.0936, loss: 0.00000761
Epoch: [35] [  24/  40] time: 363.3295, loss: 0.00000226
Epoch: [35] [  25/  40] time: 363.5671, loss: 0.00000309
Epoch: [35] [  26/  40] time: 3

Epoch: [38] [  30/  40] time: 396.0133, loss: 0.00000427
Epoch: [38] [  31/  40] time: 396.2494, loss: 0.00000067
Epoch: [38] [  32/  40] time: 396.4866, loss: 0.00000259
Epoch: [38] [  33/  40] time: 396.7227, loss: 0.00001352
Epoch: [38] [  34/  40] time: 396.9647, loss: 0.00000043
Epoch: [38] [  35/  40] time: 397.2009, loss: 0.00000050
Epoch: [38] [  36/  40] time: 397.4373, loss: 0.00000313
Epoch: [38] [  37/  40] time: 397.6746, loss: 0.00000136
Epoch: [38] [  38/  40] time: 397.9125, loss: 0.00000283
Epoch: [38] [  39/  40] time: 398.1507, loss: 0.00000171
[38/50] - ptime: 10.0379 loss: 0.00000667 acc: 0.78000
Epoch: [39] [   0/  40] time: 399.3360, loss: 0.00001365
Epoch: [39] [   1/  40] time: 399.5722, loss: 0.00006602
Epoch: [39] [   2/  40] time: 399.8080, loss: 0.00001357
Epoch: [39] [   3/  40] time: 400.0464, loss: 0.00000373
Epoch: [39] [   4/  40] time: 400.2835, loss: 0.00000167
Epoch: [39] [   5/  40] time: 400.5208, loss: 0.00003024
Epoch: [39] [   6/  40] time: 400

Epoch: [42] [  10/  40] time: 432.8766, loss: 0.00000903
Epoch: [42] [  11/  40] time: 433.1154, loss: 0.00001204
Epoch: [42] [  12/  40] time: 433.3517, loss: 0.00000008
Epoch: [42] [  13/  40] time: 433.5884, loss: 0.00000812
Epoch: [42] [  14/  40] time: 433.8249, loss: 0.00000069
Epoch: [42] [  15/  40] time: 434.0639, loss: 0.00000008
Epoch: [42] [  16/  40] time: 434.3002, loss: 0.00002404
Epoch: [42] [  17/  40] time: 434.5362, loss: 0.00005591
Epoch: [42] [  18/  40] time: 434.7734, loss: 0.00001186
Epoch: [42] [  19/  40] time: 435.0120, loss: 0.00001261
Epoch: [42] [  20/  40] time: 435.2481, loss: 0.00000521
Epoch: [42] [  21/  40] time: 435.4838, loss: 0.00000189
Epoch: [42] [  22/  40] time: 435.7199, loss: 0.00000672
Epoch: [42] [  23/  40] time: 435.9588, loss: 0.00000022
Epoch: [42] [  24/  40] time: 436.1976, loss: 0.00000141
Epoch: [42] [  25/  40] time: 436.4336, loss: 0.00000573
Epoch: [42] [  26/  40] time: 436.6704, loss: 0.00001815
Epoch: [42] [  27/  40] time: 4

Epoch: [45] [  31/  40] time: 469.0477, loss: 0.00000348
Epoch: [45] [  32/  40] time: 469.2833, loss: 0.00000110
Epoch: [45] [  33/  40] time: 469.5205, loss: 0.00002307
Epoch: [45] [  34/  40] time: 469.7568, loss: 0.00000004
Epoch: [45] [  35/  40] time: 469.9946, loss: 0.00000157
Epoch: [45] [  36/  40] time: 470.2306, loss: 0.00001341
Epoch: [45] [  37/  40] time: 470.4668, loss: 0.00000103
Epoch: [45] [  38/  40] time: 470.7041, loss: 0.00000039
Epoch: [45] [  39/  40] time: 470.9419, loss: 0.00000137
[45/50] - ptime: 10.0159 loss: 0.00000427 acc: 0.78000
Epoch: [46] [   0/  40] time: 472.1169, loss: 0.00000027
Epoch: [46] [   1/  40] time: 472.3528, loss: 0.00000225
Epoch: [46] [   2/  40] time: 472.5924, loss: 0.00000358
Epoch: [46] [   3/  40] time: 472.8300, loss: 0.00000292
Epoch: [46] [   4/  40] time: 473.0710, loss: 0.00000182
Epoch: [46] [   5/  40] time: 473.3078, loss: 0.00000194
Epoch: [46] [   6/  40] time: 473.5450, loss: 0.00000072
Epoch: [46] [   7/  40] time: 473

Epoch: [49] [  11/  40] time: 505.9855, loss: 0.00000304
Epoch: [49] [  12/  40] time: 506.2237, loss: 0.00000365
Epoch: [49] [  13/  40] time: 506.4611, loss: 0.00000091
Epoch: [49] [  14/  40] time: 506.6986, loss: 0.00000049
Epoch: [49] [  15/  40] time: 506.9358, loss: 0.00011351
Epoch: [49] [  16/  40] time: 507.1723, loss: 0.00000250
Epoch: [49] [  17/  40] time: 507.4083, loss: 0.00000098
Epoch: [49] [  18/  40] time: 507.6461, loss: 0.00000064
Epoch: [49] [  19/  40] time: 507.8844, loss: 0.00000062
Epoch: [49] [  20/  40] time: 508.1222, loss: 0.00000042
Epoch: [49] [  21/  40] time: 508.3590, loss: 0.00000053
Epoch: [49] [  22/  40] time: 508.5949, loss: 0.00000730
Epoch: [49] [  23/  40] time: 508.8319, loss: 0.00001184
Epoch: [49] [  24/  40] time: 509.0701, loss: 0.00000057
Epoch: [49] [  25/  40] time: 509.3091, loss: 0.00000002
Epoch: [49] [  26/  40] time: 509.5463, loss: 0.00002365
Epoch: [49] [  27/  40] time: 509.7826, loss: 0.00003353
Epoch: [49] [  28/  40] time: 5

In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放